In [10]:
import os, glob
import pandas as pd
from io import StringIO
import pybedtools
print(pybedtools.__file__)
pybedtools.helpers.set_tempdir('/home/pdutta/temp')

/home/pdutta/anaconda3/envs/GDC_VCF/lib/python3.10/site-packages/pybedtools/__init__.py


In [2]:
base_path = "/home/pdutta/Rekha_LabWork/Enhancer_Prediction_Project/2023/Work/Biosample_wise_models/Processing_for_DNABERT/DNABERT_fine-tune_data/cell_line/K562/NEW_data/All_CART-with-ZBTB33-enh/"

In [3]:
df_train =pd.read_csv(base_path+"K562_All_CART-with-ZBTB33-enh_train.tsv", sep="\t")
df_test =pd.read_csv(base_path+"K562_All_CART-with-ZBTB33-enh_dev.tsv", sep="\t")

In [4]:
df = pd.concat([df_train, df_test])
df_enhancer = df[df['label']==1][['coord']] 
df_enhancer

,coord
0,chr12:120523107-120523306
1,chr12:68390558-68390757
3,chr15:65604351-65604550
6,chr19:561375-561574
9,chr7:99046680-99046879
...,...
2153,chr18:2944504-2944703
2154,chr7:8267692-8267891
2156,chr6:35247125-35247324
2157,chr15:75093906-75094105


In [5]:
# Split the string at the colon (':')
df_enhancer['chromosome_name'] = df_enhancer['coord'].str.split(':').str[0]
df_enhancer['range'] = df_enhancer['coord'].str.split(':').str[1]

# Split the range at the hyphen ('-')
df_enhancer['enhancer_start'] = df_enhancer['range'].str.split('-').str[0].astype(int)
df_enhancer['enhancer_end'] = df_enhancer['range'].str.split('-').str[1].astype(int)
df_enhancer = df_enhancer.drop(columns=['coord', 'range'])
df_enhancer

,chromosome_name,enhancer_start,enhancer_end
0,chr12,120523107,120523306
1,chr12,68390558,68390757
3,chr15,65604351,65604550
6,chr19,561375,561574
9,chr7,99046680,99046879
...,...,...,...
2153,chr18,2944504,2944703
2154,chr7,8267692,8267891
2156,chr6,35247125,35247324
2157,chr15,75093906,75094105


In [8]:
str_io_enhancer = StringIO()
df_enhancer[['chromosome_name','enhancer_start','enhancer_end']].to_csv(str_io_enhancer, sep='\t', index=False, header=False)
str_io_enhancer.seek(0)
enhancer_bed = pybedtools.BedTool(str_io_enhancer.read(), from_string=True)

In [11]:
chr_wise_DBSNP_files = glob.glob('/home/pdutta/DBSNP_Chrwise/*.csv')

In [12]:
for dbsnp_file in chr_wise_DBSNP_files:
    # Read the csv file into a DataFrame
    print(dbsnp_file)
    df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])
    df = df.iloc[1:]
    print(df.head())
    
    # Convert the DataFrame to a BedTool
    dbsnp = pybedtools.BedTool.from_dataframe(df)
    
    # Intersect the BedTool with the transcript file
    intersected = enhancer_bed.intersect(dbsnp, wa=True, wb=True)
    
    # Save the intersected BedTool to a bed file
    output_file = "/home/pdutta/Data/EnhancerPoster/ZBTB33/Intersected_files/intersected_DBSNP_transcript_" + dbsnp_file.split('/')[-1].replace('.csv', '.tsv')
    intersected.saveas(output_file)

/home/pdutta/DBSNP_Chrwise/chr17_data.csv


/home/pdutta/temp/ipykernel_196946/1816505842.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0      1      2             3  4  5   6
1  chr17  60054  60055  rs2039333033  G  1  A,
2  chr17  60086  60087  rs2039333047  C  1  T,
3  chr17  60102  60103  rs1399099657  G  1  A,
4  chr17  60103  60104  rs1363626035  C  1  T,
5  chr17  60149  60150  rs2039333080  C  1  A,
/home/pdutta/DBSNP_Chrwise/chrY_data.csv


/home/pdutta/temp/ipykernel_196946/1816505842.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3  4  5     6
1  chrY  10001  10002  rs1226858834  T  1    A,
2  chrY  10002  10003   rs375039031  A  2  C,G,
3  chrY  10006  10007  rs1422184628  C  1    G,
4  chrY  10007  10008  rs1179917603  T  1    G,
5  chrY  10008  10009   rs565284081  A  2  C,G,
/home/pdutta/DBSNP_Chrwise/chr14_data.csv


...

       0         1         2             3  4  5   6
1  chr22  10510050  10510051  rs2061858963  T  1  C,
2  chr22  10510054  10510055  rs2061858972  A  1  G,
3  chr22  10510076  10510077  rs1290354662  C  1  A,
4  chr22  10510090  10510091  rs2061858975  T  1  A,
5  chr22  10510104  10510105  rs1325858619  T  1  A,
/home/pdutta/DBSNP_Chrwise/chr7_data.csv


/home/pdutta/temp/ipykernel_196946/1816505842.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3  4  5   6
1  chr7  10001  10002  rs1583156258  T  1  C,
2  chr7  10002  10003  rs1583156262  A  1  G,
3  chr7  10003  10004  rs1583156265  A  1  C,
4  chr7  10004  10005  rs1583156268  C  1  A,
5  chr7  10006  10007  rs1781584841  C  1  T,
/home/pdutta/DBSNP_Chrwise/chr13_data.csv


/home/pdutta/temp/ipykernel_196946/1816505842.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0         1         2             3  4  5   6
1  chr13  16000025  16000026  rs1175737345  T  1  A,
2  chr13  16000063  16000064  rs1957610123  G  1  A,
3  chr13  16000079  16000080  rs1957610132  G  1  T,
4  chr13  16000081  16000082  rs1957610136  A  1  C,
5  chr13  16000100  16000101  rs1957610144  T  1  C,


In [13]:
tsv_files = [f for f in os.listdir("/home/pdutta/Data/EnhancerPoster/ZBTB33/Intersected_files/") if f.endswith('.tsv') and "chrM" not in f]

In [14]:
# Read each .tsv file into a pandas DataFrame and store them in a list
list_of_dfs = []

for file in tsv_files:
    file_path = os.path.join("/home/pdutta/Data/EnhancerPoster/ZBTB33/Intersected_files", file)
    try:
        # Attempt to read the .tsv file
        df = pd.read_csv(file_path, sep='\t', header=None)
        print(df.shape)
        list_of_dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"Warning: {file_path} is empty and will be skipped.")

(22603, 10)
(17487, 10)
(16111, 10)
(6998, 10)
(35455, 10)
(24862, 10)
(9009, 10)
(13710, 10)
(22710, 10)
(17876, 10)
(15024, 10)
(5074, 10)
(36457, 10)
(21090, 10)
(17298, 10)
(23317, 10)
(6270, 10)
(9148, 10)
(32897, 10)
(53578, 10)
(18191, 10)
(6371, 10)
(8256, 10)


In [15]:
# Concatenate all the DataFrames in the list into a single DataFrame
final_df = pd.concat(list_of_dfs, ignore_index=True) if list_of_dfs else pd.DataFrame()
final_df=final_df.drop(columns=[3])
new_columns = ['chr_name', 'Enhancer_start', 'Enhancer_end', 'Variant_start', 'Variant_end', 'RS_ID', 'Reference_Nucleotide', 'No_of_variations', 'Alternative_Nucleotides']
final_df.columns = new_columns
final_df

,chr_name,Enhancer_start,Enhancer_end,Variant_start,Variant_end,RS_ID,Reference_Nucleotide,No_of_variations,Alternative_Nucleotides
0,chr19,561375,561574,561375,561376,rs1345678776,C,1,"T,"
1,chr19,561375,561574,561377,561378,rs570139519,C,2,"G,T,"
2,chr19,561375,561574,561378,561379,rs1022722361,G,1,"A,"
3,chr19,561375,561574,561380,561381,rs1980794075,C,1,"T,"
4,chr19,561375,561574,561382,561383,rs1980794126,G,2,"A,C,"
...,...,...,...,...,...,...,...,...,...
439787,chr18,2944504,2944703,2944683,2944687,rs2077422449,TTTT,1,"TTTTT,"
439788,chr18,2944504,2944703,2944688,2944689,rs2077422509,C,1,"T,"
439789,chr18,2944504,2944703,2944689,2944690,rs1271925579,A,1,"T,"
439790,chr18,2944504,2944703,2944695,2944696,rs1334793346,G,1,"A,"


In [16]:
# Write the final DataFrame to a new .tsv file (only if there's data)
if not final_df.empty:
    output_file_path = "/home/pdutta/Data/EnhancerPoster/ZBTB33/Intersected_files/all_Enhancer_data_Consensus.tsv"
    final_df.to_csv(output_file_path, sep='\t', index=False)
    print(f"All .tsv files (excluding chrM) concatenated into {output_file_path}")
else:
    print("No data to save after excluding empty files and chrM.")

All .tsv files (excluding chrM) concatenated into /home/pdutta/Data/EnhancerPoster/ZBTB33/Intersected_files/all_Enhancer_data_Consensus.tsv
